# Scripts

In this notebook we present all possible scripts.

In [1]:
from mammos_mmag.simulation import Simulation

sim = Simulation()

We load a cube enclosed with a sphere. This sphere is embedded in a further spherical shell.
The `cube.krn` file contains three lines: the first line is the cube, with nonzero values; the other two lines represent those two regions, with all zero values.

In [2]:
sim.mesh_path = "data/cube.fly"
sim.materials.read_yaml("data/materials.yaml")
sim.parameters.read("data/parameters.yaml")

When we run a script we can specify the optional variables `outdir` and `name`. While the first identifies the output directory where the input and output files will be store (and where the script is executed), the `name` variable defines the output file names.

## Show the mesh and the materials

To create the `vtk` file for the visualisation of the material properties we can use the script `materials`.

In [3]:
sim.run_materials(outdir="out/materials", name="cube")

materials:
read mesh from cube.fly


## Compute the magnetostatic field

To create the `vtk` file for visualisation of the magnetic scalar potential and the magnetic field we use the script `hmag`.
With linear basis function for the magnetic scalar potential $u$, the magnetostatic field $h = - \nabla u$ is defined at the finite elements. By smoothing the field can be transfered to the nodes of the finite element mesh. This is `h_at_nodes`.

The software also gives the magnetostatic energy density computed with finite elements and compares it with the analytic solution. 
Three energy values are compared:
- `from field`: $\frac{1}{2} \int \frac{h m}{V}$, where $V$ is the volume.
- `from_gradient`: $\frac{1}{2} \sum_i m_i \cdot g_i$, where $m_i$ and $g_i$ are the unit vector of the magnetization and the gradient of the energy normalized by the volume of the energy with respect to $m_i$ at the nodes of the finite element mesh.
- `analytic`: $J_s^2 / (6 \mu_0)$

In [4]:
sim.run_hmag(outdir="out/hmag", name="cube")

read mesh from cube.fly
apply spherical shell transformation
create uniformly magnetised state


magnetostatic energy density of uniformly magnetized cube
from field    (J/m^3) 323030.6036230678
from gradient (J/m^3) 323030.6036230672
analytic      (J/m^3) 343787.94003580953


## exchange and anisotropy energy

To test the computation of the exchange and anisotropy energy density we can use the script `exani`.

This gives the exchange energy density of a vortex in the $xy$-plane and the anistropy energy density in the uniformly magnetized state.
Here we have placed the anistropy direction paralle to to the $z$-axis. The anisotropy energy density is calculated as $-K (m \cdot k)^2$  where $m$ is the unit vector of magnetization and $k$ is the anisotropy direction. $K$ is the magnetocrystalline anisotropy constant

In [5]:
sim.run_exani(outdir="out/exani", name="cube")

read mesh from cube.fly


exchange energy density of a vortex on a, b plane
energy from gradient (J/m^3) 983624.0554480732
analytic             (J/m^3) 986960.4401089362
create uniformly magnetised state


magnetocrystalline anisotropy energy density of uniformly
magnetized sample in direction [0.0, 0.0, 1.0]
from gradient (J/m^3) -4900000.000646964
analytic      (J/m^3) -4900000.00064696


## Zeeman energy

The script `external` calculates the Zeeman energy density in the external field by finite elements and analytically.

In [6]:
sim.run_external(outdir="out/external", name="cube")

tolerances: optimality tolerance 1e-10   hmag 1e-10
read mesh from cube.fly
create uniformly magnetised state


Zeeman energy density for an external field
of mu_0 Hext  (T)     1.2
from gradient (J/m^3) -1537202.6523594826
analytic      (J/m^3) -1537202.6523594814


## jax implementation

The above tools checked the energy calculation with the finite element backend. 
From the finite element backend system matrices are generated for micromagnetic simulations. 
The script `mapping` is used to test the energy calculations with matrices.

The `mmag` software uses sparse matrix methods from `jax`.

In [7]:
sim.run_mapping(outdir="out/mapping", name="cube")



MAP FINITE ELEMENT BACKEND (esys-escript) TO JAX 

Memory before escript2jax        183.1015625 MB
tolerances: optimality tolerance 1e-10   hmag 1e-10
read mesh from cube.fly
create uniformly magnetised state
apply spherical shell transformation

Zeeman energy density for an external field
of mu_0 Hext     (T)     1.2
with jax backend (J/m^3) -1537201.6
analytic         (J/m^3) -1537202.6523594814

exchange energy density of a vortex on a, b plane
with jax backend (J/m^3) 983624.3
analytic         (J/m^3) 986960.4401089362

magnetocrystalline anisotropy energy density of uniformly
magnetized sample in direction [0.0, 0.0, 1.0]
from gradient (J/m^3) -4899996.0
analytic      (J/m^3) -4900000.00064696

magnetostatic energy density of uniformly magnetized cube
with jax backend (J/m^3) 323030.47
analytic         (J/m^3) 343787.94003580953


Memory after  escript2jax        454.28125 MB
Memory after  garbage collection 454.28125 MB


COMPUTE TOTAL ENERGY

--> timing and statistics
elaplsed

## Storing sparse matrices

The sparse matrices used for computation can be stored and reused for simulations with the same finite element mesh. To store the matrices use the script `store`.

In [8]:
sim.run_store(outdir="out/store", name="cube")

tolerances: optimality tolerance 1e-10   hmag 1e-10
read mesh from cube.fly
create uniformly magnetised state
apply spherical shell transformation


## Demagnetization curve - Hysteresis Loop

To compute the demagnetization curve we use the script `loop`.

This creates the file `cube.dat` which gives the demagnetization curve. The columns of the file are:
- `vtk number`: the number of the `vtk` file that corresponds to the field and magnetic polarisation values in the line.
- `mu0 Hext`: the value of $\mu_0 H_{\mathsf{ext}}$ ($T$) where $\mu_0$ is the permability of vacuum and $H_{\mathsf{ext}}$ is the external value of the external field.
- `polarisation`: the componenent of magnetic polarisation ($T$) parallel to the direction of the external field.
- `energy density`: the energy density ($J/m^3$) of the current state.

In [9]:
sim.run_loop(outdir="out/loop", name="cube")

Memory before conversion to jax  182.97265625 MB
tolerances: optimality tolerance 1e-10   hmag 1e-10
read mesh from cube.fly
create uniformly magnetised state
apply spherical shell transformation
Memory after  conversion to jax  448.1640625 MB
Memory after  garbage collection 448.1640625 MB
--> demag 1.0 1.6096814901982122
--> demag 0.8 1.6096688701022317
--> demag 0.6000000000000001 1.6096554325587062
--> demag 0.4000000000000001 1.609641101269194
--> demag 0.20000000000000007 1.6096257900457391
--> demag 5.551115123125783e-17 1.6096094032620407
--> demag -0.19999999999999996 1.6095918323404783
--> demag -0.39999999999999997 1.6095729541997779
--> demag -0.6 1.6095526287879989
--> demag -0.8 1.6095306961070197
--> demag -1.0 1.6095069726689015

--> done
elaplsed time             89.08133602142334
iterations                32
  function calls          75
